In [1]:
import pandas as pd
from konlpy.tag import Okt
from numpy import ndarray
import warnings
import re 
from model.bow import BagOfWords
from model.tfidf import TfIdf
import yaml
warnings.filterwarnings("ignore")

C:\Users\gnsdl\AppData\Local\Temp\ipykernel_8904\2874203659.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# config
with open('configs/database.yaml') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
from utils.query_service import QueryService
query_service = QueryService(cfg['mysql'])

In [4]:
query_service.query_to_pandas_df('Show tables')

,Tables_in_bakery
0,board_preference_statistic
1,board_statistic
2,food_survey
3,image
4,member
5,member_preference
6,notice
7,preference
8,product
9,product_board


In [5]:
PRODUCT_BOARD_SQL = """
SELECT id
, store_id
, title
, price
, is_soldout
, view 
FROM product_board
"""
product_board = query_service.query_to_pandas_df(PRODUCT_BOARD_SQL)

In [6]:
PRODUCT_SQL = """
SELECT id
, product_board_id
, title as option
, price as option_price
, category
, gluten_free_tag
, high_protein_tag
, sugar_free_tag
, vegan_tag
, ketogenic_tag
FROM product
"""
product = query_service.quaery_to_pandas_df(PRODUCT_SQL)

AttributeError: 'QueryService' object has no attribute 'quaery_to_pandas_df'

In [ ]:
REVIEW_SQL = """
SELECT board_id
, badge_taste
, badge_brix
, badge_texture
, rate
, content
FROM review
"""
review = query_service.query_to_pandas_df(REVIEW_SQL)
review = review.drop_duplicates(subset=['board_id', 'content'], keep='first')

In [ ]:
BOARD_STAT_SQL = """
SELECT board_id
, board_review_grade
, board_view_count
, board_review_count
, board_wish_count 
FROM board_statistic
"""
board_stat = query_service.query_to_pandas_df(BOARD_STAT_SQL)

In [7]:
# board_id가 각 preference_type마다 얼만큼의 preference_score를 갖는지
# 아직 신뢰성있는 데이터로 보이진 않음 (샘플이 적고 score의 분산이 너무 큼)
BOARD_PREFERENCE_STAT_SQL = """
SELECT board_id
, preference_score
, preference_type
, preference_weight
FROM board_preference_statistic
ORDER BY board_id
"""
board_preference_stat = query_service.query_to_pandas_df(BOARD_PREFERENCE_STAT_SQL)

#### Data Import

In [8]:
df = pd.read_csv('data/product_board_feature.csv', encoding='utf-8')

In [9]:
# NLP 
okt = Okt()
def tokenize(text):
    return ['/'.join(t) for t in okt.pos(text, norm=True, stem=True)]

# 자연어 데이터만 추출
nl_df = df[['id', 'title', 'flavor']]

In [10]:
nl_df['title'] = nl_df['title'].astype(str)
nl_df['flavor'] = nl_df['flavor'].astype(str)

#### Preprocessing

In [11]:
# 소문자로 변환
nl_df['title'] = nl_df['title'].str.lower()
nl_df['flavor'] = nl_df['flavor'].str.lower()

In [12]:
# 특수문자 제거
special_chars = '[^가-힣0-9a-zA-Z]'
nl_df['title'] = nl_df['title'].apply(lambda x: re.sub(special_chars, ' ', x))
nl_df['flavor'] = nl_df['flavor'].apply(lambda x: re.sub(special_chars, ' ', x))

In [13]:
# preprocessing Rule
# 목적 : token이 될 수 있는 유효한 단어만 남기기 위해 불필요한 단어의 종류를 정의하고 제거한다.
# 수량 제거
quantity_compiler = re.compile(r'\d+[개입|kg|팩|개|종|입|원|g]+')
nl_df['title'] = nl_df['title'].apply(lambda x: re.sub(quantity_compiler, ' ', x))
nl_df['flavor'] = nl_df['flavor'].apply(lambda x: re.sub(quantity_compiler, ' ', x))

In [14]:
# title과 option을 합쳐서 description을 만든다.
nl_df['description'] = nl_df['title'] + ' ' + nl_df['flavor'] 

In [15]:
# tokenizing
nl_df['token'] = nl_df['description'].apply(lambda x: okt.morphs(x))

In [16]:
# 중복 단어 제거 : 나열식의 중복 단어가 노이즈로 인식됨 
nl_df['token'] = nl_df['token'].apply(lambda x: list(set(x)))

In [17]:
# stopwords 제거
with open('data/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
    stopwords = [x.strip() for x in stopwords]
nl_df['token'] = nl_df['token'].apply(lambda x: [word for word in x if word not in stopwords])

In [18]:
# description mutation : 
nl_df['description'] = nl_df['token'].apply(lambda x: ' '.join(x))

In [19]:
# id2idx
id2idx = {id:idx for idx, id in enumerate(nl_df['id'])}
idx2id = {idx:id for idx, id in enumerate(nl_df['id'])}

In [20]:
board_id_2_store_id = {board_id:store_id for board_id, store_id in zip(df['id'], df['store_id'])}

In [21]:
description = nl_df.set_index('id').to_dict()['title']
# import json
# with open('data/description.json', 'w', encoding='utf-8') as f:
#     json.dump(description, f)

#### Fit
- 현재 있는 모델은 tfidf, bow입니다.
- 아래의 예시처럼 model.fit()에 description 컬럼을 넣는다고 생각하시면 됩니다.

In [22]:
tfidf = TfIdf()
tfidf.fit(nl_df['description'])
tfidf.predict(1)

[(18, 0.40174844125052417),
 (27, 0.2529894330230058),
 (64, 0.2420079329174314),
 (85, 0.18994660095218632),
 (54, 0.16220861905396078),
 (143, 0.15885571639989043),
 (141, 0.15235309711767359),
 (206, 0.14804650603794528),
 (0, 0.14365131675828186),
 (30, 0.1435167876245107)]

In [23]:
bow = BagOfWords()
bow.fit(nl_df['description'])
bow.predict(1)

[(18, 0.483045891539648),
 (27, 0.4409585518440984),
 (85, 0.4123930494211613),
 (125, 0.3779644730092272),
 (64, 0.36313651960128146),
 (141, 0.3504383220252312),
 (162, 0.3450327796711771),
 (55, 0.3380617018914066),
 (17, 0.329914439536929),
 (155, 0.329914439536929)]

#### Generate

In [24]:
def generate_recommendation(board_id: int, cosine_sim: ndarray=None, top_k: int=10, threshold: float=0.5) -> list:
    """
    1. 추천을 생성한다
    2. thresholding을 한다 
    3. 같은 스토어 상품을 제외한다
    4. topk를 뽑는다
        - 안되면 예외처리한다
    :params:
        id : int : product id
        cosine_sim : ndarray : cosine similarity matrix
        top_k : int : number of recommendation
        threshold : float : similarity threshold
    :return:
    """
    idx = id2idx[board_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # score 기준으로 정렬
    sim_scores = [i for i in sim_scores if i[1] > threshold]
    sim_scores = sim_scores[1:top_k+1] 
    candidate = [(board_id, idx2id[i], score) for i, score in sim_scores] # id, score 형태로 변환 
    product_indices = [i[0] for i in sim_scores]
    # print(product_indices)
    print('Query Item:\n ', df[['title', 'purchase_url']].iloc[idx])
    print('Recommendation Items:\n', df[['title', 'purchase_url']].iloc[product_indices], '\n', '------------------------')
    return candidate

In [21]:
result = generate_recommendation(board_id=15, cosine_sim=tfidf.cosine_matrix, top_k=3)

Recommendation Items:
         title  \
124  비건버터 쌀식빵   

                                                         purchase_url  
124  https://smartstore.naver.com/dearmyqueen1553/products/9022144231  


In [26]:
result = generate_recommendation(board_id=5, cosine_sim=bow.cosine_matrix, top_k=3)

Query Item:
  title                                         꾸디 비건쿠키 (르뱅쿠키 수제쿠키)
purchase_url    https://smartstore.naver.com/ggudi/products/53...
Name: 4, dtype: object
Recommendation Items:
                          title  \
166      스모어쿠키 르뱅 수제 아메리칸 비건쿠키   
17   레몬 얼그레이 수제쿠키 쌀 글루텐프리 비건쿠키   

                                          purchase_url  
166  https://smartstore.naver.com/veganez/products/...  
17   https://smartstore.naver.com/veganez/products/...   
 ------------------------


#### Postprocessing
- 같은 스토어의 제품은 나오지 않게 한다
- 품절된 상품은 나오지 않게 한다

In [27]:
board_id_2_store_id = {board_id:store_id for board_id, store_id in zip(df['id'], df['store_id'])}
def store_exist(board_id:int, recommended_store_id_set: set)->bool:
    store_id = board_id_2_store_id.get(board_id, None)
    return store_id in recommended_store_id_set

from collections import defaultdict
# boardid가 같은 storeid를 갖고있는지 확인
recommend_dict = defaultdict(list)
recommended_store_id_set = set()
for board_id, recommendation_board_id, score in result:
    recommended_store_id = board_id_2_store_id.get(recommendation_board_id, None)
    if len(recommend_dict[board_id]) < 3:
        if not store_exist(recommendation_board_id, recommended_store_id_set):
            recommended_store_id_set.add(recommended_store_id)
            recommend_dict[board_id].append((recommendation_board_id, score))

#### Metrics

In [36]:
from metrics.coverage import get_coverage
from collections import defaultdict 
rec_result = defaultdict(list)
total_cnt = nl_df['id'].nunique()
rec_list = []
for board_id in nl_df['id']:
    result = generate_recommendation(board_id, bow.cosine_matrix, 3)
    rec_result[board_id].extend(result)
    rec_list.extend([x[1] for x in result])
rec_set = set(rec_list)
get_coverage(rec_set, total_cnt)

Query Item:
  title                            냥빵 파운드 저탄수 대체당 제로 빵 글루텐프리 건강빵 비건
purchase_url    https://smartstore.naver.com/nyangppang/produc...
Name: 0, dtype: object
Recommendation Items:
                             title  \
135              비건 글루텐프리 베이글 [쑥]   
42   카카오 큐브 대체당 냥빵 글루텐프리 대체당 노밀가루   
26              비건 글루텐프리 베이글 [호박]   

                                          purchase_url  
135  https://smartstore.naver.com/dearmyqueen1553/p...  
42   https://smartstore.naver.com/nyangppang/produc...  
26   https://smartstore.naver.com/dearmyqueen1553/p...   
 ------------------------
Query Item:
  title                        비건 유기농 쌀 & 기장쌀 잡곡 비스코티 5종 미앤드리 글루텐프리
purchase_url    https://smartstore.naver.com/meandri/products/...
Name: 1, dtype: object
Recommendation Items:
 Empty DataFrame
Columns: [title, purchase_url]
Index: [] 
 ------------------------
Query Item:
  title                       비건이레 비건케이크 글루텐프리 떠먹는케이크 얼그레이아몬드베린느 GF
purchase_url    https://smartstore.naver.com/veg

0.6066350710900474

In [29]:
from collections import Counter
agg_cnt = Counter(rec_list)

In [50]:
total_rows = 0
matched_rows = 0
for board_id, rec_list in rec_result.items():
    total_rows += len(rec_list)
    for rec in rec_list:
        if rec[0] == rec[1]:
            matched_rows += 1
            print(rec)

(112, 112, 1.0)
(122, 122, 1.0000000000000002)
(178, 178, 0.9999999999999998)
(209, 209, 1.0000000000000002)
(211, 211, 0.9999999999999999)


In [51]:
rec_result

defaultdict(list,
            {1: [(1, 136, 0.6154574548966638),
              (1, 43, 0.6030226891555273),
              (1, 27, 0.5698028822981898)],
             2: [],
             3: [],
             4: [],
             5: [(5, 167, 0.7302967433402215), (5, 18, 0.50709255283711)],
             6: [(6, 29, 0.7378647873726218),
              (6, 139, 0.5892556509887895),
              (6, 45, 0.5555555555555556)],
             7: [(7, 30, 0.5039526306789696)],
             8: [(8, 9, 0.5773502691896258),
              (8, 114, 0.5555555555555556),
              (8, 209, 0.5555555555555556)],
             9: [(9, 76, 0.6666666666666669),
              (9, 98, 0.6666666666666669),
              (9, 188, 0.6390096504226939)],
             10: [(10, 69, 0.7161148740394329),
              (10, 25, 0.5962847939999439),
              (10, 183, 0.5773502691896257)],
             11: [(11, 190, 0.801783725737273)],
             12: [],
             13: [(13, 176, 0.6227991553292184),
       

In [30]:
agg_df = pd.DataFrame.from_dict(agg_cnt, orient='index').reset_index()
agg_df.columns = ['id', 'count']
fig = plt.figure(figsize=(10, 6))
plt.bar(agg_df['id'], agg_df['count'])

NameError: name 'plt' is not defined